In [3]:
import logging, time, sys, decimal, OpenOPC
from datetime import datetime
from opcua import ua, uamethod, Server

In [4]:
## 1. Configuration
OPC_DA_SERVER = 'Matrikon.OPC.Simulation.1'
OPC_UA_CERTIFICATE = 'C:/Users/BUGER/Desktop/ca.der'
OPC_UA_PRIVATE_KEY = '7bccbe83b08d5bb8bd40c069a73e1b51a9778cee'
OPC_UA_URI = 'opc.tcp://127.0.0.1:49310'

In [5]:
# Constants
ITEM_ID_VIRTUAL_PROPERTY = 0
ITEM_CANONICAL_DATATYPE = 1
ITEM_VALUE = 2
ITEM_QUALITY = 3
ITEM_TIMESTAMP = 4
ITEM_ACCESS_RIGHTS = 5
SERVER_SCAN_RATE = 6
ITEM_EU_TYPE = 7
ITEM_EU_INFO = 8
ITEM_DESCRIPTION = 101
ACCESS_READ = 0
ACCESS_WRITE = 1
ACCESS_READ_WRITE = 2

In [6]:
  opc = OpenOPC.client() 

In [7]:
opc.servers()

[u'Intellution.IntellutionGatewayOPCServer', u'Matrikon.OPC.Simulation.1']

In [8]:
opc.connect('Matrikon.OPC.Simulation.1')

In [9]:
opc.info()

[('Protocol', 'DCOM'),
 ('Class', 'Matrikon.OPC.Automation'),
 ('Client Name', u'OpenOPC'),
 ('OPC Host', 'DESKTOP-IHBHDGT'),
 ('OPC Server', u'Matrikon.OPC.Simulation.1'),
 ('State', 'Running'),
 ('Version', '1.7 (Build 7433)'),
 ('Browser', 'Hierarchical'),
 ('Start Time', '12/05/19 15:32:34'),
 ('Current Time', '12/05/19 15:32:36'),
 ('Vendor', u'Matrikon Inc +1-780-945-4011 http://www.matrikonopc.com')]

In [10]:
opc.read('test.AI')

(None, 'Error', None)

In [103]:
server.stop()

In [105]:
server = Server()
server.set_endpoint("opc.tcp://192.168.2.32:48400/freeopcua/server/")
server.start()

error: [Errno 10048] error while attempting to bind on address ('192.168.2.32', 48400): 通常每个套接字地址(协议/网络地址/端口)

In [87]:
# Set up server
#server.load_certificate(OPC_UA_CERTIFICATE)
#server.load_private_key(OPC_UA_PRIVATE_KEY)
uri = OPC_UA_URI
idx = server.register_namespace(uri)
root = server.nodes.objects.add_object(idx, OPC_DA_SERVER)

In [88]:
## 2. Connect to OPC-DA server
c = OpenOPC.client()

In [85]:
class SubscriptionHandler(object):
    def __init__(self,n):
        self.i = 0
        self.n = n
    def final_datachange_notification(self, node, val, data):
        path_as_string = node.get_path_as_string()
        # 'path_as_string' is a list of strings containing:
        # 0: 0:Root
        # 1: 1:Objects
        # 2: 2:OPC DA Server
        # 3 and onwards: 3:[Step of path to node in OPC-DA]
        opc_da_address = '.'.join([a.split(':')[1] for a in path_as_string[3:]])
        cc = OpenOPC.client()
        cc.connect(OPC_DA_SERVER)
        print('Datachange', opc_da_address, val, cc.write((opc_da_address, val,)))
        cc.close()
    # This function is called initially to catch the notifications from newly added nodes
    def datachange_notification(self, node, val, data):
        self.i = self.i + 1
        #print('Catching meaningless datachange notification')
        if self.i == self.n:
            #print('Finished catching meaningless datachange notifications')
            self.datachange_notification = self.final_datachange_notification

def read_value(value):
    value = value[0]
    if isinstance(value,decimal.Decimal):
        value = float(value)
    elif isinstance(value,list):
        if len(value) == 0:
            value = None
    elif isinstance(value,tuple):
        if len(value) == 0:
            value = None
    return value

## 3. Discover OPC-DA server nodes
cc = OpenOPC.client()
readable_variable_handles = {}
writeable_variable_handles = {}
nodes = cc.list('*',recursive=True)
# 'nodes' is a list of dot-delimited strings.
tree = {}
for node in nodes:
    parts = node.split('.')
    # Folders are the steps on the path to the file.
    folders = parts[:-1]
    file = parts[-1]
    # Create folder tree if it does not already exist
    for i, folder in enumerate(folders,1):
        if i == 1:
            parent = root
        else:
            parent = tree[path]
        path = '.'.join(folders[0:i])
        if path not in tree.keys():
            tree[path] = parent.add_folder(idx,folder)
    # 'path' is now the folder that file resides in.
    # Determine node properties
    node_obj = {}
    for id, description_of_id, value in c.properties(node):
        if id is ITEM_ACCESS_RIGHTS:
            if value == 'Read':
                value = ACCESS_READ
            elif value == 'Write':
                value = ACCESS_WRITE
            elif value == 'Read/Write':
                value = ACCESS_READ_WRITE
        node_obj[id] = value
    current_value = read_value((node_obj[ITEM_VALUE],))
    if type(current_value) != int:
        current_value = 0
    #print('Adding node '+file+' at path '+path)
    opcua_node = tree[path].add_variable(idx, file, ua.Variant(current_value, ua.VariantType.UInt16))
    # Determine readable vs. writable
    if node_obj[ITEM_ACCESS_RIGHTS] in [ACCESS_READ]:
        readable_variable_handles[node] = opcua_node
    if node_obj[ITEM_ACCESS_RIGHTS] in [ACCESS_WRITE, ACCESS_READ_WRITE]:
        opcua_node.set_writable()
        writeable_variable_handles[node] = opcua_node

try:
    server.start()
    ## 4. Subscribe to datachanges coming from OPC-UA clients
    handler = SubscriptionHandler(len(writeable_variable_handles))
    sub = server.create_subscription(100, handler).subscribe_data_change(writeable_variable_handles.values())
    readables = list(readable_variable_handles.keys())
    while True:
        time.sleep(0.5)
        ## 5. Read all readables simultaneously and update the OPC-UA variables
        for reading in c.read(readables):
            opc_da_id = reading[0]
            variable_handle = readable_variable_handles[opc_da_id]
            variable_handle.set_value(read_value(reading[1:]))
finally:
    server.stop()
    cc.close()

AttributeError: 'NoneType' object has no attribute 'call_soon_threadsafe'